In [1]:
!cat .env

STAGE=CCR_HOMOLOG


In [2]:
company = Company.objects.get(name='CCR - Rio Sp')
company

<Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>

In [3]:
ApprovalFlow.objects.get(company=company, target_model="daily_reports.MultipleDailyReport")

<ApprovalFlow: [CCR - Rio Sp]: Fluxo RDO - daily_reports.MultipleDailyReport>

In [4]:
for permission in UserPermission.objects.filter(companies=company).exclude(name='Inativo'):
    
    
    try:
        if 'MultipleDailyReport' in permission.permissions.keys():
            permission.permissions['MultipleDailyReport']['can_approve'] = True
            print(permission)
        else:
            permission.permissions['multiple_daily_report']['can_approve'] = True
            print(permission)
    except Exception as e:
#         print(f'Não possui parametrização - {permission}')
        pass

    permission.save()

['CCR - AutoBAn', 'CCR - RodoAnel', 'CCR - SPVias', 'CCR - Rio Sp'] - Suporte
['CCR - AutoBAn', 'CCR - RodoAnel', 'CCR - SPVias', 'CCR - Rio Sp'] - Fiscal
['CCR - AutoBAn', 'CCR - RodoAnel', 'CCR - SPVias', 'CCR - Rio Sp'] - Coordenador
['CCR - AutoBAn', 'CCR - RodoAnel', 'CCR - SPVias', 'CCR - Rio Sp'] - Administrador
['CCR - AutoBAn', 'CCR - RodoAnel', 'CCR - SPVias', 'CCR - Rio Sp'] - Terceiro (campo)
['CCR - AutoBAn', 'CCR - RodoAnel', 'CCR - SPVias', 'CCR - Rio Sp'] - Terceiro (escritório)
['CCR - AutoBAn', 'CCR - RodoAnel', 'CCR - SPVias', 'CCR - Rio Sp'] - Usuário padrão
['CCR - AutoBAn', 'CCR - RodoAnel', 'CCR - SPVias', 'CCR - Rio Sp'] - Sala técnica operacional
['CCR - AutoBAn', 'CCR - RodoAnel', 'CCR - SPVias', 'CCR - Rio Sp'] - Núcleo técnico
['CCR - AutoBAn', 'CCR - RodoAnel', 'CCR - SPVias', 'CCR - Rio Sp'] - Coordenador/Gerente
['CCR - AutoBAn', 'CCR - RodoAnel', 'CCR - SPVias', 'CCR - Rio Sp'] - Inspetor
['CCR - AutoBAn', 'CCR - RodoAnel', 'CCR - SPVias', 'CCR - Rio Sp'

In [22]:
flow = ApprovalFlow(
    name="Fluxo RDO",
    target_model="daily_reports.MultipleDailyReport",
    company=company
)
flow.save()

In [23]:
elaboracao = ApprovalStep(
    name="RDO Criado",
    approval_flow=flow
)
elaboracao.responsible_created_by = True
elaboracao.save()
elaboracao.responsible_users.add(User.objects.get(username='rlcs'))
# elaboracao.responsible_firms.add(Firm.objects.get(name='Programação e Controle', company=company))
# elaboracao.responsible_firms.add(Firm.objects.get(name='Suporte', company=company))

In [24]:
homolog_normal = ApprovalStep(
    name="RDO Aguardando Aprovação",
    approval_flow=flow,
)
homolog_normal.save()
homolog_normal.responsible_users.add(User.objects.get(username='rlcs'))
# homolog_normal.responsible_firms.add(Firm.objects.get(name='Programação e Controle', company=company))
# homolog_normal.responsible_firms.add(Firm.objects.get(name='Suporte', company=company))

In [25]:
deferido = ApprovalStep(
    name="RDO Aprovado",
    approval_flow=flow,
)
deferido.save()

In [26]:
trans_homolog_normal = ApprovalTransition(
    name="Solicitar aprovação",
    origin=elaboracao,
    destination=homolog_normal,
    condition={
        "===": [{
            "var": "request.action"
            }, "Solicitar aprovação"]
    },
    button={"icon": "assignment", "color": "primary", "variant": "raised"}
)
trans_homolog_normal.save()

In [27]:
trans_homologar_normal = ApprovalTransition(
    name="Aprovar",
    origin=homolog_normal,
    destination=deferido,
    condition={
        "===": [{
            "var": "request.action"
        }, "Aprovar"] 
    },
    button={"icon": "check", "color": "primary", "variant": "raised"},
    callback={"change_fields": [{"name": "editable", "value": False}]}
)
trans_homologar_normal.save()

In [28]:
elaboracao=ApprovalStep.objects.get(approval_flow__company=company, name='RDO Criado')
elaboracao

<ApprovalStep: [Arteris Intervias]: RDO Criado>

In [29]:
MultipleDailyReport.objects.filter(company=company).update(approval_step=elaboracao)

57

In [30]:
str(deferido.uuid)

'74c453e6-5b87-47d0-a922-7d1e9868a121'

In [31]:
company.metadata['approved_approval_steps'].append(str(deferido.uuid))

In [32]:
company.save()

In [33]:
homolog_normal.responsible_firms.add(*Firm.objects.filter(company=company,name='Aprovadores de RDO'))

In [34]:
homolog_normal.responsible_firms.all()

<QuerySet [<Firm: [b8450161-14d1-4de8-b254-7d1ddc4a9c20: Arteris Intervias] 9234a29c-1edf-4b08-8e01-913d54dd61b5: Aprovadores de RDO>]>

# Incluir em fluxo existente

In [4]:
flow=ApprovalFlow.objects.get(
    name="Fluxo RDO",
    target_model="daily_reports.MultipleDailyReport",
    company=company
)

In [5]:
homolog_normal = ApprovalStep.objects.get(
    name="RDO Aguardando Aprovação",
    approval_flow=flow,
)

In [6]:
elaboracao = ApprovalStep.objects.get(
    name="RDO Criado",
    approval_flow=flow
)

In [7]:
trans_revisao = ApprovalTransition(
    name="Solicitar Revisão",
    origin=homolog_normal,
    destination=elaboracao,
    condition={
        "===": [{
            "var": "request.action"
        }, "Solicitar Revisão"] 
    },
    button={"icon": "warning", "color": "amber", "variant": "outlined", "needs_dialog": True},
    callback={"change_fields": [{"name": "editable", "value": True}]}
)
trans_revisao.save()